In [1]:
import os
from experiment_setup import setups
from model_tracker import track_training_C02_emissions

In [2]:
# download the dataset, the links can be fund in the README
dataset = "yoochoose"
dataset_path = f"../datasets/{dataset}"
model_path = "../trained_models"

## Run the preprocess script, specific to the dataset you chose

- The preprocessing script in general, executes the following steps:
    - Loads the raw data, with correct types
    - Creates the sessions
    - Removes duplicated items. An item is considered as a duplicate if the preceding (based on time) event in the same session contains the exact same item.
    - Performes iterative support filtering
        - Removes sessions with only one event
        - Removes items with less than 5 events
        - Until the size of the dataset changes


In [ ]:
%run ../Preprocess/coveo_preproc.py --path $dataset_path

## Use a specific setup for your dataset

In [3]:
params = setups[dataset]["params_xe"]

In [4]:
train_path = os.path.join(dataset_path,f"{dataset}_processed_view_train_full.tsv")
test_path = os.path.join(dataset_path,f"{dataset}_processed_view_test.tsv")

In [5]:
def create_gru4rec_tensorflow_script(model_name, train_path, test_path, model_path, loss, final_act, layers, batch_size, dropout_p_hidden, learning_rate, hidden_act, n_epochs, m, decay, initial_accumulator):
        """Creates the train and test script.
        """
        s_train_full = (
                f"python ../GRU4Rec_TensorFlow/gru4rec_BP/main.py --train_path {train_path} "
                f"--test_path {test_path} --checkpoint_dir {model_path}/{model_name} --layer {1} --size {layers} "
                f"--epoch {n_epochs} --lr {learning_rate} --hidden_act {hidden_act} --final_act {final_act} "
                f"--loss {loss} --dropout {dropout_p_hidden} --batch_size {batch_size} "
                f"--initial_accumulator_value {initial_accumulator} --decay {decay}"
        )
        s_test_full = s_train_full + f" --train 0 --test {n_epochs-1} --m {m}"
        return s_train_full, s_test_full

In [6]:
loss = params["loss"]
optim = params["optim"]
const_emb = params["constrained_embedding"]
embed = params["embedding"]
final_act = params["final_act"]
layers = params["layers"]
batch_size = params["batch_size"]
dropout_p_embed = params["dropout_p_embed"]
dropout_p_hidden = params["dropout_p_hidden"]
learning_rate = params["learning_rate"]
momentum = params["momentum"]
sample_alpha = params["sample_alpha"]
bpreg = params["bpreg"]
logq = params["logq"]
hidden_act = params["hidden_act"]
n_epochs = 5
m = '1 5 10 20'

## Train & test (minor fix model)


In [10]:
train_script_minor, test_script_minor = create_gru4rec_tensorflow_script(model_name='gru4rec_tensorflow_minorfix', train_path=train_path, test_path=test_path, model_path=model_path, loss=loss, final_act=final_act, layers=layers, batch_size=batch_size, dropout_p_hidden=dropout_p_hidden, learning_rate=learning_rate, hidden_act=hidden_act, n_epochs=n_epochs, m=m, decay=1.0, initial_accumulator=1e-12)

### Train the minor fix model

In [8]:
from model_tracker import track_training_C02_emissions

track_training_C02_emissions(train_script_minor, "gru4rec_tensorflow_minorfix", "yoochoose")

[codecarbon WARNING @ 09:20:38] Invalid gpu_ids format. Expected a string or a list of ints.
[codecarbon INFO @ 09:20:38] [setup] RAM Tracking...
[codecarbon INFO @ 09:20:38] [setup] GPU Tracking...
[codecarbon INFO @ 09:20:40] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 09:20:40] [setup] CPU Tracking...
[codecarbon WARNING @ 09:20:40] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 09:20:41] We saw that you have a 13th Gen Intel(R) Core(TM) i9-13900HX but we don't know it. Please contact us.
[codecarbon INFO @ 09:20:41] CPU Model on constant consumption mode: 13th Gen Intel(R) Core(TM) i9-13900HX
[codecarbon INFO @ 09:20:41] >>> Tracker's metadata:
[codecarbon INFO @ 09:20:41]   Platform system: Windows-11-10.0.22631-SP0
[codecarbon INFO @ 09:20:41]   Python version: 3.12.3
[codecarbon INFO @ 09:20:41]   CodeCarbon version: 2.4.2
[codecarbon INFO @ 09:20:41]   Available RAM : 31.746 GB
[codecarbon INFO @ 09:20:41]   CPU count: 32
[codecarbon 

Salida de STDOUT:                          Args                                         Values
0                     n_items                                          34858
1                      layers                                              1
2                    rnn_size                                            480
3                    n_epochs                                              5
4               learning_rate                                           0.07
5                 is_training                                              1
6                  test_model                                              2
7                  hidden_act                                           tanh
8                   final_act                                        softmax
9                        loss                                  cross-entropy
10                 batch_size                                             48
11                      decay                             

1.2332969079004532

### Test the minor fix model

In [9]:
print(test_script_minor)
# os.system(test_script_minor)

python ../GRU4Rec_TensorFlow/gru4rec_BP/main.py --train_path ../datasets/rees46\rees46_processed_view_train_full.tsv --test_path ../datasets/rees46\rees46_processed_view_test.tsv --checkpoint_dir ../trained_models/gru4rec_tensorflow_minorfix --layer 1 --size 512 --epoch 5 --lr 0.065 --hidden_act tanh --final_act softmax --loss cross-entropy --dropout 0.0 --batch_size 240 --initial_accumulator_value 1e-12 --decay 1.0 --train 0 --test 4 --m 1 5 10 20


0